In [ ]:
import tensorflow as ts
import tensorflow.keras
from tensorflow.keras.datasets import mnist

#Loading the data into train and test sets

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## The data is 28x28 pixel handwriting digits images (0 - 9)

In [ ]:
print(f'X Train shape: {x_train.shape} \n ')

print(f'X Test shape: {x_test.shape} \n ')


## Flattening  the image to a 1x784 vector

In [ ]:
image_vector_size = 28*28
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)

##Making the lables from numrical to a categorical ones (good for cost functuon and tranig)

In [ ]:
from tensorflow.keras.utils import to_categorical
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
pip install scikeras 

In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential # Basic model wich on top of we'll add layares 
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout # A danse layer obcjet that we add to Sequential model
from scikeras.wrappers import KerasClassifier 

##Building a model and tuning it's hyperparameters as folow:
####1. Build a genral model functuon "model_generator" 
####2. Add layer/s with pramas as var with the ".add()" method
####3. Build a GridSearchCV/Random object and pass it the params
####4. Fit the data(traind) with .fit() method.
####5. Check the scores 

In [ ]:
import random
import itertools
import time
from keras.callbacks import EarlyStopping
def model_generator(neurons,activation,optimizer,dropout_rate,norm=False,regularization=False):
  model = Sequential()
  if norm:
    if regularization:
      for i in range(len(neurons)):
        model.add(Dense(neurons[i],activation=activation[0]))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    else:
        for i in range(len(neurons)):
          model.add(Dense(neurons[i],activation=activation[0]))
          model.add(BatchNormalization())
  elif regularization:
        for i in range(len(neurons)):
          model.add(Dense(neurons[i],activation=activation[0]))
          model.add(Dropout(dropout_rate))
  else:
    for i in range(len(neurons)):
 
      model.add(Dense(neurons[i],activation=activation[0]))

  model.add(Dense(10,activation=activation[1]))
  model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
  return model

def MyRandomSearchCV(neurons: list,activation:list,optimizer:list,norm=False,regularization=False,epochs=30,batch_size=100,dropout_rate=0.2,early_stop=False,model=False):
    random_indexs_neutrons = tuple(set([random.randint(0,len(neurons) -1 ) for i in range(int(len(neurons) * 0.1 + 1))]))
    random_indexs_activation = tuple(set([random.randint(0,len(activation) - 1) for i in range(int(len(activation) * 0.1 + 1))]))
    random_indexs_optimizer = tuple(set([random.randint(0,len(optimizer) - 1) for i in range(int(len(optimizer) * 0.1 + 1))]))
    list_of_list = [random_indexs_neutrons,random_indexs_activation,random_indexs_optimizer]
    all_combintation = tuple((itertools.product(*list_of_list)))
    accuracy_map = {}
    model_map = {}
    for combination in all_combintation:
    
      neurons_index,activation_index,optimizer_index = combination
      model = model_generator(neurons=neurons[neurons_index],
                              activation=activation[activation_index],optimizer=optimizer[optimizer_index],
                              norm=norm,regularization=regularization,dropout_rate=dropout_rate)
      if early_stop:
        es = EarlyStopping(monitor='val_loss', mode='min',patience=1, verbose=1)
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1, verbose=True,callbacks=[es])
      else:
        model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=.1, verbose=False)
      loss, accuracy  = model.evaluate(x_test, y_test, verbose=False)
      key = (tuple(neurons[neurons_index]),tuple(activation[activation_index]),optimizer[optimizer_index])
      model_map[key] = {'accuracy' : accuracy,'loss' : loss, 'model':model}
      
    return model_map


## Adam v.s SGD

In [ ]:
import time
T = time.time()
adam_ac = 0
num_of_iter = 1 
neurons = [(60,30,15)]
activations = [('relu','sigmoid')]
optimizer_adam = ['adam']
optimizer_sgd = ['sgd']

for i in range(num_of_iter):
  adam_ac += MyRandomSearchCV(neurons,activations,optimizer_adam)[(neurons[0],activations[0],optimizer_adam[0])]['accuracy']
print(f'Adam\'s AVG traning time is {(time.time() - T)/num_of_iter}\n')
print(f'Adam\'s AVG accuracy is {adam_ac/num_of_iter}\n')
T = time.time()
sgd_ac = 0
for i in range(num_of_iter):
  sgd_ac +=  MyRandomSearchCV(neurons,activations,optimizer_sgd)[(neurons[0],activations[0],optimizer_sgd[0])]['accuracy']
print(f'SGD\'sAVG traning time is {(time.time() - T)/num_of_iter}\n')
print(f'SGD\'s AVG accuracy is {sgd_ac/num_of_iter}\n')

## Now let's use batch normalization

In [ ]:
T = time.time()
norm_ac = 0
num_of_iter = 1
for i in range(num_of_iter):
  norm_ac += MyRandomSearchCV(neurons,activations,optimizer_adam ,norm=True)[(neurons[0],activations[0],optimizer_adam[0])]['accuracy']
print(f'Adam\'s AVG traning time is {(time.time() - T)/num_of_iter}\n')
print(f'Adam\'s AVG accuracy is {(norm_ac)/num_of_iter}\n')



## Now let's Add dropout regularization with norm 

In [ ]:
T = time.time()
reg_ac = 0
num_of_iter = 1
for i in range(num_of_iter):
  reg_ac += MyRandomSearchCV(neurons,activations,optimizer_adam,norm=True,regularization=True)[(neurons[0],activations[0],optimizer_adam[0])]['accuracy']
print(f'Adam\'s AVG traning time is {(time.time() - T)/num_of_iter}\n')
print(f'Adam\'s AVG accuracy is {(reg_ac)/num_of_iter}\n')

## Only regularization

In [ ]:
T = time.time()
reg_ac1 = 0
num_of_iter = 1
neurons = [(90,60,30,15)]
for i in range(num_of_iter):
  reg_ac1 += MyRandomSearchCV(neurons,activations,optimizer_adam,regularization=True,dropout_rate=0.25)[(neurons[0],activations[0],optimizer_adam[0])]['accuracy']
print(f'Adam\'s AVG traning time is {(time.time() - T)/num_of_iter}\n')
print(f'Adam\'s AVG accuracy is {(reg_ac1)/num_of_iter}\n')

##Early stopage
####Want to see when the acuurcy is not changing significantly and stop

In [ ]:
from keras.callbacks import EarlyStopping
T = time.time()
num_of_iter = 1
accuracy = 0
neurons = [(60,30,15)]
for i in range(num_of_iter):
 res = MyRandomSearchCV(neurons,activations,optimizer_adam,early_stop=True)[(neurons[0],activations[0],optimizer_adam[0])]
 accuracy += res['accuracy']
 model = res['model']
print(f"Accuracy = {accuracy/num_of_iter}")
print(f"Time = {time.time() - T}")

## We'll define two functions that will give us more metrics and visualize it

In [ ]:
from keras.callbacks import EarlyStopping
import time
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
def get_categories(model,y_test):

  pred = model.predict(x_test)
  y_hat = []
  y = []
  for i in range(len(pred)):
    y_hat.append(list(pred[i]).index(max(pred[i])))
    y.append(list(y_test[i]).index(max(y_test[i])))
  return y, y_hat

def conf_matrix(model,y_test,font=0.8,fig=(10,10)):
  y, y_hat = get_categories(model,y_test)
  cm=confusion_matrix(y,y_hat)
  report = classification_report(y,y_hat) 
  df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
  plt.figure(figsize = fig)
  sns.set(font_scale=font)
  hm = sns.heatmap(df_cm, annot=True)
  print(hm)
  print(report)
  return y,y_hat



In [ ]:
y,y_hat=conf_matrix(model=model,y_test=y_test)

## Let's see the actual pics the model got wrong

In [ ]:
from matplotlib import pyplot as plt
def get_pics(test_val,pred_val,num_of_pics,y,y_hat):
  (x_train2, y_train2), (x_test2, y_test2) = mnist.load_data()
  data = [x_test2[i] for i in range(len(y)) if y[i] == test_val and y_hat[i] == pred_val]
  for d in range(num_of_pics):
    plt.imshow(data[d], interpolation='nearest')
    plt.show()


In [ ]:
get_pics(7,1,10,y,y_hat)

## Fix it with more layers and neurons
### Note: it did better on some and worst on other

In [ ]:
neurons = [(90,60,45,30,15)]
activations = [('relu','sigmoid')]
optimizer_adam = ['adam']
res = MyRandomSearchCV(neurons,activations,optimizer_adam,early_stop=True)[(neurons[0],activations[0],optimizer_adam[0])]
accuracy = res['accuracy']
model = res['model']
conf_matrix(model,y_test)
print(f"Time :\n {time.time() - T}")

## Traying the softmax activation function inested of sigmoid

In [ ]:
neurons=[(45,30,15)]
activations = [('relu','softmax')]
res = MyRandomSearchCV(neurons,activations,optimizer_adam,early_stop=True)[(neurons[0],activations[0],optimizer_adam[0])]
accuracy = res['accuracy']
model = res['model']
conf_matrix(model,y_test)
print(f"Time :\n {time.time() - T}")